# LAB05 Computation of standard redox potentials
(from https://education.molssi.org/qm-tools/05-redox/index.html)

## Questions
* How can you calculate the redox potential of a reaction?

## Objectives
* Perform geometry optimization calculations.
* Perform frequency calculations.
* Calculate the free energy correction from the frequencies.


## Overview
One of the most important research problems in chemistry is to find efficient ways to accomplish reactions such as CO2 reduction or water splitting that make chemical fuels from electricity. If this could be accomplished efficiently and inexpensively, we would be one step further on the way to a renewable energy economy.

The redox potential is a molecular property in chemistry that is very important for renewable energy problems. It measures the tendency of a species to gain or lose electrons in solution. In order to apply computational chemistry to renewable energy problems, being able to compute redox potentials for molecular species is incredibly important.

In order to compute a redox potential, it is important to understand the basic physical principles of this quantity. For example, take this half-reaction that involves the nitrobenzene molecule:

![image.png](redox1.png "redox1")

This redox potential measures the tendency of nitrobenzene to gain an electron to produce nitrobenzene radical anion. More negative values of Eº mean that the species is less likely to gain electrons, or that it requires more energy to reduce. To make a comparison, consider this redox half-reaction that involves acetophenone:

![image.png](redox2.png "redox2")

The more negative Eº shows that acetophenone requires more energy to reduce compared to nitrobenzene. Alternatively, acetophenone radical anion would spontaneously transfer an electron to nitrobenzene if they were combined. In this process, the Gibbs free energy of the reaction is obtained by subtracting the redox potentials and multiplying by the number of electrons, in this case, ∆G = –0.73 eV per molecule, or –17 kcal/mol.

Going back to our individual half-reactions, the redox potential is related to the Gibbs free energy of the half reaction as:

![image.png](redox3.png "redox3")

Here, Eº is the redox potential corresponding to the quantity that the experiment measures. This is the potential (voltage) of an electrode surface where reactant and product are in equilibrium. More negative potentials correspond to higher energy electrons needed to reduce the reactant.

G is the free energy change of the reduction half-reaction in solution, where the electron comes from the vacuum, and the molecules are in solution. n is the number of electrons transferred (in our case, n = 1) and F is the Faraday constant; its value is exactly 1 eV/V and serves to convert units of energy (in eV) into units of potential (V).

The electrons in the experiment do not come from vacuum, but from an electrode whose potential is measured relative to a reference electrode. We choose to use the normal hydrogen electrode as a reference, where the energy of the electrons is –4.43 V relative to the vacuum, thus we subtract this value from our answer.

Now that we have seen the physical principles, we’d like to obtain computationally and this would allow us to predict a redox potential. is the Gibbs free energy of reduction, and can be computed as a free energy difference: where G(●) is the Gibbs free energy of a species in solution. Computationally, this is broken down into several contributions:

The first two terms and represent the ground-state electronic energy plus the solvation free energy term. These two terms are computed together as the ground-state energy when running a Psi4 energy calculation in implicit solvent. The equilibrium (minimum energy) structure should be used, but you don’t necessarily need to use implicit solvent, or even the same functional or basis set to perform the minimization, as long as you remain consistent for the oxidized and reduced species.

H – TS represent the enthalpy and entropy terms in the Gibbs free energy. We will approximate these terms by treating the molecule as having three translational and three rotational degrees of freedom, and represent all of the internal motions using harmonic oscillators. The translational contributions to H and S come from known formulas for the ideal gas, the rotational contributions use a rigid rotor approximation that takes information from the molecular geometry, and the vibrational contribution requires a harmonic frequency calculation (including the zero point vibration). The frequency analysis calculation in Psi4 provides all of this information.

## Summary of the calculation procedure
In summary, here are the steps for computing the redox potential:

1. For the neutral molecule, follow these steps to compute the Gibbs free energy:
    * Calculate an energy minimized structure.
    * Calculate the harmonic free energy correction at the energy minimized structure.
    * Calculate the ground state energy plus ∆Gsolv from implicit solvent. (Part (c) may be done at a higher level of theory than (a) and (b)).
    * Add the results from (b) and (c) to get the free energy of the species.
2. Repeat the above steps for the anion radical.
3. Calculate ∆Gred = G(A–) – G(A). This should be a negative number because most molecules in solvent are able to accommodate an additional electron.
4. Convert this energy to eV by multiplying your quantity (in a.u.) by 27.211.
5. Multiply by –1 and subtract 4.43 to obtain the redox potential w/r.t. NHE.

In this exercise, we will study neutral nitrobenzene and the nitrobenzene anion to calculate the redox potential.

### Part A: Initial molecule specification and optimizations

In [1]:
import psi4
import numpy as np

In [2]:
psi4.set_memory('2 GB')
psi4.set_num_threads(2)

nitrobenz = psi4.geometry("""
symmetry c1
0 1
C       -3.5048225421    0.0711805817    0.1456897967
C       -2.1242069042    0.0676957680    0.1437250554
C       -1.4565144627    1.2657898054    0.0112805274
C       -2.1243502782    2.4616659201   -0.1394727314
C       -3.5049153121    2.4578370923   -0.1457245349
C       -4.1936081427    1.2645153194    0.0001955136
H       -4.0381801262   -0.8505059514    0.2559173303
H       -1.5620288767   -0.8346363876    0.2389155097
H       -1.5619534389    3.3630228735   -0.2428628637
H       -4.0382012347    3.3785626398   -0.2639829256
H       -5.2650389640    1.2641688843   -0.0022762561
N       -0.0085078655    1.2648596634   -0.0056641832
O        0.5639468379    0.1670702678   -0.1297708787
O        0.5668300231    2.3598431617    0.1306822195
""")

psi4.set_output_file('00_initial_energy.dat', False)
psi4.energy('B3LYP/3-21G')

-434.31176489184566

### Part B: Geometry optimization (gas phase)

In [3]:
psi4.set_output_file('01_geometry_optimization.dat', False)
psi4.set_options({'g_convergence': 'gau_tight'}) # this forces the optimizer to get close to the minimum
psi4.optimize('B3LYP/3-21G', molecule=nitrobenz)

Optimizer: Optimization complete!


-434.3175493752826

### Part C: Frequency calculation

In [4]:
psi4.set_output_file('02_frequency_analysis.dat', False)
b3lyp_321g_energy, b3lyp_321g_wfn = psi4.frequency('B3LYP/3-21G', molecule=nitrobenz, return_wfn=True, dertype='gradient')

 73 displacements needed.
 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73
 '498.9109']


We have to define a function called `parse_free_energy` to extract the free energy from the output file.

In [5]:
# define a function called parse_free_energy to extract the free energy from the output file

def parse_free_energy(filename):
    """ Parse out the free energy from a Psi4 vibrational analysis output file in a.u. """
    for line in open(filename).readlines():
        if "Correction G" in line:
            return float(line.split()[-2])

In [6]:
parse_free_energy('02_frequency_analysis.dat')

0.07134802

### Part D: Energy calculation with large basis set + PCM solvent model

In [9]:
import pcmsolver

psi4.set_options({
  'pcm': True,
  'pcm_scf_type': 'total',
})

psi4.pcm_helper("""
   Units = Angstrom
   Medium {
   SolverType = CPCM
   Solvent = Acetonitrile
   }
   Cavity {
   RadiiSet = UFF
   Type = GePol
   Scaling = False
   Area = 0.3
   Mode = Implicit
   }
""")

psi4.set_output_file('03_solvent_energy.dat', False)
psi4.energy('B3LYP/heavy-aug-cc-pVDZ', molecule=nitrobenz)

ModuleNotFoundError: No module named 'pcmsolver'

### Exercise: The nitrobenzene anion

Using the commands you already learned, calculate all the same quantities for the reduced nitrobenzene (-1 charge). In your molecule specification section, you will need to change your charge and multiplicity to reflect the correct values for the anion. If you already carried out calculations on the neutral molecule in this notebook, you will need to start a new notebook or click Kernel -> Restart & Clear Output before working on the anion. For the anion calculations, you need to change the refernce using the command `psi4.set_options({'reference':'UKS'})` which should be placed right before the energy call.

In [ ]:
psi4.set_memory('2 GB')
psi4.set_num_threads(2)

# Anion
nitrobenzA = psi4.geometry("""
symmetry c1
-1 2
C       -3.5048225421    0.0711805817    0.1456897967
C       -2.1242069042    0.0676957680    0.1437250554
C       -1.4565144627    1.2657898054    0.0112805274
C       -2.1243502782    2.4616659201   -0.1394727314
C       -3.5049153121    2.4578370923   -0.1457245349
C       -4.1936081427    1.2645153194    0.0001955136
H       -4.0381801262   -0.8505059514    0.2559173303
H       -1.5620288767   -0.8346363876    0.2389155097
H       -1.5619534389    3.3630228735   -0.2428628637
H       -4.0382012347    3.3785626398   -0.2639829256
H       -5.2650389640    1.2641688843   -0.0022762561
N       -0.0085078655    1.2648596634   -0.0056641832
O        0.5639468379    0.1670702678   -0.1297708787
O        0.5668300231    2.3598431617    0.1306822195
""")

psi4.set_options({'reference':'UKS'}
psi4.set_output_file('10_initial_energy.dat', False)
psi4.energy('B3LYP/3-21G')

In [ ]:
psi4.set_output_file('10_geometry_optimization.dat', False)
psi4.set_options({'g_convergence': 'gau_tight'}) # this forces the optimizer to get close to the minimum
psi4.optimize('B3LYP/3-21G', molecule=nitrobenzA)

In [ ]:
psi4.set_output_file('20_frequency_analysis.dat', False)
b3lyp_321g_energy, b3lyp_321g_wfn = psi4.frequency('B3LYP/3-21G', molecule=nitrobenzA, return_wfn=True, dertype='gradient')

In [ ]:
parse_free_energy('20_frequency_analysis.dat')

In [ ]:
psi4.set_options({
  'pcm': True,
  'pcm_scf_type': 'total',
})

psi4.pcm_helper("""
   Units = Angstrom
   Medium {
   SolverType = CPCM
   Solvent = Acetonitrile
   }
   Cavity {
   RadiiSet = UFF
   Type = GePol
   Scaling = False
   Area = 0.3
   Mode = Implicit
   }
""")

psi4.set_output_file('30_solvent_energy.dat', False)
psi4.energy('B3LYP/heavy-aug-cc-pVDZ', molecule=nitrobenzA)

### Solution

Your final results should be as follows.

Charge on nitrobenzene	| Minimized E (gas) | Free energy correction (gas)|	SCF E + Delta G Solvation	| Total Gibbs Free Energy
--------|--------|--------|--------|--------
0 (oxidized)	 |-434.317549 |	0.071346	 |-436.813669	 |-436.74232254
-1 (reduced)	| -434.338874 |	0.068022	 |-436.942923	 |-436.874901
 	 	 	 	 
To calculate E standard

∆Gred = -436.874901+436.74232254 = -0.13257846 a.u. = -3.607645 eV

-∆Gred / nF = 3.607645 V

Eº = -∆Gred / nF - 4.43 V = -0.82 V

### Key Points

* The overall redox potential energy should be between -1 and -10 eV.
* The free energy of a radical anion should be more negative than the neutral molecule.

In [ ]:
# now for the case of acetophenone
psi4.set_memory('2 GB')
psi4.set_num_threads(2)

acetophenone = psi4.geometry("""
symmetry c1
0 1
 C                 -0.00000100    1.07431000   -0.00005100
 O                 -0.00000200    2.31943700   -0.00009300
 C                 -1.29543300    0.32464600    0.02052200
 C                 -1.44515300   -0.95854900    0.56779600
 C                 -2.42954600    1.00709600   -0.44683000
 C                 -2.70396500   -1.56099700    0.60733300
 H                 -0.58746100   -1.46824000    0.98753900
 C                 -3.68093600    0.40021100   -0.42186100
 H                 -2.29805300    2.01506400   -0.82040700
 C                 -3.81965300   -0.88976900    0.10208800
 H                 -2.81497700   -2.54851100    1.03984900
 H                 -4.54834900    0.92886200   -0.79936400
 H                 -4.79422300   -1.36358800    0.12750400
 C                  1.29543200    0.32464500   -0.02055900
 C                  1.44515800   -0.95859500   -0.56772500
 C                  2.42954200    1.00713600    0.44674400
 C                  2.70397000   -1.56104600   -0.60720000
 H                  0.58747000   -1.46832000   -0.98743500
 C                  3.68093200    0.40024800    0.42183500
 H                  2.29804400    2.01513400    0.82023700
 C                  3.81965400   -0.88977600   -0.10200400
 H                  2.81498500   -2.54859700   -1.03963300
 H                  4.54834400    0.92893100    0.79929900
 H                  4.79422400   -1.36359700   -0.12737200
""")

psi4.set_output_file('00_initial_energy.dat', False)
psi4.energy('B3LYP/3-21G')

In [ ]:
psi4.set_memory('2 GB')
psi4.set_num_threads(2)

# Anion
acetophenoneA = psi4.geometry("""
symmetry c1
-1 2
 C                  0.00000100    0.97687000    0.00002000
 O                  0.00000000    2.27740700   -0.00001000
 C                 -1.28963400    0.27392800    0.01177200
 C                 -1.53286700   -1.08176100    0.36531500
 C                 -2.43762100    1.07003200   -0.26843700
 C                 -2.82291500   -1.60772800    0.37221300
 H                 -0.71115700   -1.70541200    0.69098400
 C                 -3.71623700    0.53862700   -0.26954600
 H                 -2.24521200    2.11578800   -0.47358500
 C                 -3.93161600   -0.81720700    0.03894000
 H                 -2.97000400   -2.64656900    0.65804900
 H                 -4.56426000    1.17709900   -0.50599100
 H                 -4.93263500   -1.23645000    0.03830400
 C                  1.28963400    0.27392800   -0.01175200
 C                  1.53286100   -1.08176200   -0.36530000
 C                  2.43762700    1.07003000    0.26844200
 C                  2.82290900   -1.60773000   -0.37221800
 H                  0.71114600   -1.70541100   -0.69095800
 C                  3.71624300    0.53862400    0.26953000
 H                  2.24522100    2.11578500    0.47359500
 C                  3.93161500   -0.81721000   -0.03896100
 H                  2.96999200   -2.64657000   -0.65805700
 H                  4.56427000    1.17709500    0.50596400
 H                  4.93263400   -1.23645400   -0.03834000
""")

psi4.set_options({'reference':'UKS'}
psi4.set_output_file('10_initial_energy.dat', False)
psi4.energy('B3LYP/3-21G')